In [0]:
!nvidia-smi

Thu Jun 11 23:53:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [0]:
!pip install transformers

In [0]:
# Load SQuAD Dataset
from transformers.data.processors.squad import SquadResult, SquadV1Processor, SquadV2Processor, squad_convert_examples_to_features
import tensorflow_datasets as tfds
tfds_examples = tfds.load("squad")
examples = SquadV1Processor().get_examples_from_dataset(tfds_examples, evaluate=True)

INFO:absl:No config specified, defaulting to first: squad/plain_text
INFO:absl:Load pre-computed datasetinfo (eg: splits) from bucket.
INFO:absl:Loading info from GCS for squad/plain_text/1.0.0
INFO:absl:Generating dataset squad (/root/tensorflow_datasets/squad/plain_text/1.0.0)


INFO:absl:Downloading https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json into /root/tensorflow_datasets/downloads/rajpurkar_SQuAD-explorer_train-v1.1uLsZc14btZFRCgHMAy9Mn5abwO6wga4bMozTBvOyQAg.json.tmp.35bbfb1e6c734a7293ff84af387fe1d3...
INFO:absl:Downloading https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json into /root/tensorflow_datasets/downloads/rajpurkar_SQuAD-explorer_dev-v1.1nVRimH71-BT-FaNpwXJPbsOaIBizticanX0lmGhsov8.json.tmp.cf9c323729d34b62bcd3491ed442693c...
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urll

INFO:absl:generating examples from = /root/tensorflow_datasets/downloads/rajpurkar_SQuAD-explorer_train-v1.1NSdmOYa4KVr09_zf8bof8_ctB9YaIPSHyyOKbvkv2VU.json


Shuffling and writing examples to /root/tensorflow_datasets/squad/plain_text/1.0.0.incomplete981SVT/squad-train.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/squad/plain_text/1.0.0.incomplete981SVT/squad-train.tfrecord. Shard lengths: [87599]
INFO:absl:Generating split validation


INFO:absl:generating examples from = /root/tensorflow_datasets/downloads/rajpurkar_SQuAD-explorer_dev-v1.1lapqUtXWpzVWM2Z1PKUEkqZYAx2nTzAaxSOLA5Zpcsk.json


Shuffling and writing examples to /root/tensorflow_datasets/squad/plain_text/1.0.0.incomplete981SVT/squad-validation.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/squad/plain_text/1.0.0.incomplete981SVT/squad-validation.tfrecord. Shard lengths: [10570]
INFO:absl:Skipping computing stats for mode ComputeStatsMode.AUTO.
INFO:absl:Constructing tf.data.Dataset for split None, from /root/tensorflow_datasets/squad/plain_text/1.0.0


Dataset squad downloaded and prepared to /root/tensorflow_datasets/squad/plain_text/1.0.0. Subsequent calls will reuse this data.


10570it [00:19, 543.54it/s]


In [0]:
import torch
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
features, dataset = squad_convert_examples_to_features(
            examples=examples,
            tokenizer=tokenizer,
            max_seq_length=512,
            doc_stride=128,
            max_query_length=64,
            is_training=False,
            return_dataset="pt",
            #threads=args.threads,
)

add example index and unique id: 100%|██████████| 10570/10570 [00:00<00:00, 572921.26it/s]


In [0]:
train_dataloader = DataLoader(dataset, batch_size=1)

from transformers import BertForQuestionAnswering
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model.to(device)

In [0]:
import re
import gc

def get_scores(dataloader,model):
    dataset_scores = {}
    for idx, batch in enumerate(dataloader):
        batch = tuple(t.to(device) for t in batch)
        maxscore = float('-inf')
        batch_scores = []
        # print(idx,batch)
        inputs = {
                    "input_ids": batch[0],
                    "attention_mask": batch[1],
                    "token_type_ids": batch[2],
                    "start_positions": batch[3],
                    "end_positions": batch[4],
                }
        # print(input_ids)
        masked_inputs = inputs.copy()
        tokens = tokenizer.convert_ids_to_tokens(batch[0][0])
        # print(tokens)
        # print(tokenizer.decode(batch[0][0]))
        
        original_output = model(**inputs)
        predict_start,predict_end = original_output[1:3]

        i = 0
        while i < len(inputs['token_type_ids'][0]):
            # ONLY EVALUATE IMPORTANCE OF WORDS IN THE QUESTION
            # if inputs['token_type_ids'][0][i] == 1:
                # break
            if tokens[i] == '[SEP]': 
                break
            # if tokens[i] == '[CLS]':
            #     continue
            restore = tokens[i]
            cur_word_tokens = [tokens[i]]
            tokens[i] = '[MASK]'
            masked_ids = tokenizer.convert_tokens_to_ids(tokens)
            masked_ids = torch.tensor(masked_ids).unsqueeze(0).to(device)
            masked_inputs['input_ids'] = masked_ids
            masked_output = model(**masked_inputs)
            mask_start,mask_end = masked_output[1:3]
            score = torch.sum(predict_start-mask_start) + torch.sum(predict_end-mask_end)
            tokens[i] = restore
            i += 1
            # current token is a sub-word token, check the following suffix sub-word tokens
            while '##' in tokens[i]:
                restore = tokens[i]
                cur_word_tokens.append(tokens[i])
                tokens[i] = '[MASK]'
                masked_ids = tokenizer.convert_tokens_to_ids(tokens)
                masked_ids = torch.tensor(masked_ids).unsqueeze(0).to(device)
                masked_inputs['input_ids'] = masked_ids
                masked_output = model(**masked_inputs)
                mask_start,mask_end = masked_output[1:3]
                score += torch.sum(predict_start-mask_start) + torch.sum(predict_end-mask_end)
                tokens[i] = restore
                i += 1
            cur_word = tokenizer.convert_tokens_to_string(cur_word_tokens)
            cur_score = score / len(cur_word_tokens)
            # print(cur_word,cur_score.data)
            batch_scores.append((cur_word, cur_score.data))

        dataset_scores[idx] = batch_scores
        if idx == 50:
            break
        
        del masked_inputs
        del masked_ids
        del masked_output
        del tokens
        del original_output
        del cur_word
        del cur_score
        gc.collect()
    return dataset_scores


with torch.no_grad():
    model.eval()
    dataset_scores = get_scores(train_dataloader,model)
for s in dataset_scores.values():
    print(s)


[('[CLS]', tensor(-3.2464, device='cuda:0')), ('how', tensor(-2.9490, device='cuda:0')), ('many', tensor(-83.2799, device='cuda:0')), ('g3p', tensor(-26.8897, device='cuda:0')), ('molecules', tensor(-62.3952, device='cuda:0')), ('leave', tensor(-103.3983, device='cuda:0')), ('the', tensor(-9.8909, device='cuda:0')), ('cycle', tensor(-67.0270, device='cuda:0')), ('?', tensor(-356.5264, device='cuda:0'))]
[('[CLS]', tensor(-0.3713, device='cuda:0')), ('what', tensor(-15.6332, device='cuda:0')), ('type', tensor(-44.6381, device='cuda:0')), ('of', tensor(-16.7768, device='cuda:0')), ('antenna', tensor(-129.0674, device='cuda:0')), ('was', tensor(5.6644, device='cuda:0')), ('used', tensor(-11.5346, device='cuda:0')), ('for', tensor(-12.8282, device='cuda:0')), ('communication', tensor(-43.2131, device='cuda:0')), ('on', tensor(-8.6982, device='cuda:0')), ('the', tensor(2.5202, device='cuda:0')), ('lunar', tensor(-1.4104, device='cuda:0')), ('flights', tensor(-16.5796, device='cuda:0')), ('?